Markov chains: some work with text!

In [ ]:
import numpy as np

# first, read the text file
# change encoding to utf-8-sig, otherwise it will add a Byte order mark (BOM, a unicode symbol) in the beginning of the file!
with open('pride_and_prejudice.txt',mode='r',  encoding='utf-8-sig') as f:
    book = f.read()
    
print(book)

FileNotFoundError: [Errno 2] No such file or directory: 'pride_and_prejudice.txt'

In [ ]:
from collections import OrderedDict
import itertools

# let's remove some the text in the beginning and leave only the actual text of the book. 
normal_words = book.split()[105:]

# add dots
for i in range(len(normal_words)):
    if '.' in normal_words[i] and len(normal_words[i]) > 1:
        normal_words[i] = normal_words[i].replace(".", "")
        normal_words.insert(i+1, '.')

# and also remove some text in the end
normal_words = normal_words[:len(normal_words)-2926]

print(normal_words[len(normal_words)-10:len(normal_words)])


# define the vector of states (unique words)
# use this and not SET to preserve the order! 
states = list(OrderedDict.fromkeys(normal_words).keys())

print(len(normal_words))
print(len(states))
states


Create a dictionary where keys are the actual words and values are the integer labels

In [ ]:
import numpy as np
# enumrate the states
labs = np.arange(0, len(states))

# turn the sequence of words into a sequence of integers, and map them together into a dictionary
res = {states[i]: labs[i] for i in range(len(labs))}

print(res.values())
res.keys()


Now make a method for mapping the words into integers and back

In [ ]:
def map_words_into_integers(normal_words):
    mapped_words = []

    for cur_word in normal_words:
        cur_ind = list(res.keys()).index(cur_word)
        mapped_words.append(list(res.values())[cur_ind])
        
    return mapped_words


In [ ]:
def map_integers_into_words(integers):
    real_words = []

    for cur_word in integers:
        cur_ind = list(res.values()).index(cur_word)
        real_words.append(list(res.keys())[cur_ind])
        
    return real_words

and define a method to create transition matrix

In [ ]:
def transition_matrix(chain_seq):
    # define number of states
    n = 1+ max(chain_seq) 
    # create an empty nXn matrix, which we will fill in
    M = np.zeros((n,n))
    # zip is a useful method for working with matrices
    # for each states, we add up to a matrix whenever we find this state in the sequence 
    for (i,j) in zip(chain_seq,chain_seq[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    M = M/M.sum(axis=1, keepdims=True)
    return M

test the function: first, make integers out of words (might take a while)

In [ ]:
# test the function: first, make integers out of words (might take a while)

words_mapped = map_words_into_integers(normal_words)



Now compute the transition matrix

In [ ]:
p_words = transition_matrix(words_mapped)

# check the sum across rows, should be one!
p_words.sum(axis=1)

Generate samplings of your own text using the estimated probability matrix! 

In [ ]:
from collections import Counter

def generate_and_write(p_words, init_state, filename):

    def generate_text(states, p_words, init_state):

        # generate some text with N words
        N = 500

        # for the current word, generate the new word k times and choose the most common one
        k = 10

        n_states = len(states)
        # p_words is the matrix

        # initial state:
        i1 = init_state
        all_words = [i1]
        for i in range(1, N):
            # all words generated k times for the current state
            cur_states_word = []
            # given the current state, choose the column of the matrix accordingly 
            p_column = p_words[all_words[i-1]]
            # generate k words according to the matrix
            for j in range(k):
                cur_states_word.append(np.random.choice(np.arange(0, n_states), p = p_column))
                
            # now choose the most common word
            most_common_word = list(Counter(cur_states_word).most_common(1)[0])[0]
            all_words.append(most_common_word)
            
        return all_words


    text_integers = generate_text(states, p_words, init_state)
    my_text = map_integers_into_words(text_integers)

    whole_text = ' '.join(my_text)

    with open(filename, "w") as text_file:
        text_file.write(whole_text)

In [ ]:
# now generate text and write into a file
generate_and_write(p_words, 0, 'text-generation-attempt.txt')



In [ ]:
# print out the file:
with open('text-generation-attempt.txt', "r") as my_file:
    gen_text = my_file.read()

print(gen_text)


Dynamic plots in jupiter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
 
 
def interactive_plot(amplitude, frequency):
    x = np.linspace(0, 2 * np.pi, 10)
    y = amplitude * np.sin(frequency * x)
    plt.figure(figsize=(8, 4))
    plt.plot(x, y)
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.title('Interactive Sine Wave')
    plt.grid(True)
    plt.show()
 
 
#

In [ ]:
interact(interactive_plot, amplitude=(1, 5, 0.1), frequency=(1, 10, 0.1))


In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact
 

some_seq = [1, 2, 1, 3, 4, 10, 3, 2, 1, 6, 20, 1, 2, 3]

# make dynamic plot

def plot_seq(n):
    len_y = n #len(y) if  type(y) == list else 1
    x = np.arange(0, len_y)
    y = some_seq[0:n]
    plt.figure(figsize=(8, 4))
    plt.plot(x, y)
    plt.xlabel('n steps')
    plt.ylabel('Process state')
    plt.title('Interactive Process plot')
    plt.grid(True)
    plt.show()
 

In [ ]:
interact(plot_seq, n = np.arange(0, len(some_seq))) #[some_seq[0:1], some_seq[0:2], some_seq[0:3]])

Markov chain examples: Stationary distribuiton. Recall: pi*M = pi

In [ ]:
import numpy as np
# define some matrix of probabilities
transition_matrix = np.array([[0,0.5,0.5],
                              [0,0.3,0.7],
                              [1,0,0]])

# do the rows sum to 1?
print(transition_matrix.sum(axis=1))

transition_matrix


 No built-in method to find stationary distribution. One of the algorithms:
 Find the eigenvector with an eigenvalue that is are close to one. To compute the stationary distribution, every element in the selected eigenvector is divided by the sum of the eigenvector

In [ ]:
# transpose the matrix
transition_matrix_t = transition_matrix.T
# use linear algebra module of numpy to compute the eigenvalues
eigenvals, eigenvects = np.linalg.eig(transition_matrix_t)


# Find the indexes of the eigenvalues that are close to one.
close_to_1_idx = np.isclose(eigenvals,1) # isclose checks element-wise if each element is close to 1 within the specific tolerance
# Use them to select the target eigen vectors
target_eigenvect = eigenvects[:,close_to_1_idx]
#Flatten
target_eigenvect = target_eigenvect[:,0]
# Turn the eigenvector elements into probabilities
# that would be the stationary distribution
st_d = target_eigenvect / sum(target_eigenvect) 

In [ ]:
# get the real part of the probability
print(st_d.real)

# our stationary distribution pi
real_st_distr = st_d.real

# check if they sum to 1
st_d.real.sum()

Check the fixed point equation: pi*M = pi

In [ ]:
real_st_distr.dot(transition_matrix)
